**PRÁCTICA 4. CCNs CON KERAS PARA LA CLASIFICACIÓN DE IMÁGENES**

Despues de trabajar durante las dos prácticas anteriores con la librería de bajo nivel denominada TensorFlow, en la presente práctica se va a introducir un framework de alto nivel para el entrenamiento de redes neuronales denominado **Keras**. Esta librería fue desarrollada por **François Chollet** en 2015 con el objetivo de **simplificar la programación de algoritmos basados en aprendizaje profundo** ofreciendo un conjunto de abstracciones más intuitivas y de alto nivel. Keras hace uso de librerías de más bajo nivel o ***backend*** por detrás, concretamente se puede escoger entre **TensorFlow,  Microsoft Cognitive Toolkit o Theano**. Durante las sesiones prácticas que restan en el curso haremos uso de la librería Keras con TensorFlow como backend.

**EJERCICIO 1.** En primer lugar y con el objetivo de familiarizarnos con esta nueva librería, el primer ejercicio consistirá en replicar la última versión de la red neuronal profunda de la práctica anterior **empleando Keras**.  Si recordaís  el objetivo que perseguía la práctica anterior era el de **clasificar el dataset de dígitos manuscritos denominado MNIST**, así que vamos a ello:

- **Analiza el siguiente código con atención** y **busca en la [documentación de Keras](https://keras.io/)** cada una de las funciones que se utilizan. No pases al siguiente apartado hasta no tener **totalmente claro** cada uno de los **comandos** que se emplean y los **parámetros de entrada** de dichas funciones.

- **Ejecuta el proceso de entrenamiento de la red**. Posteriormente **añade capas de dropout (manteniendo el 50% de las neuronas de cada capa) después de cada una de las *hidden layers*** y vuelve a lanzar la red. En el código aparece comentada las instrucciones para añadir las capas de dropout. 

- Realiza de nuevo estas dos pruebas cambiando el optimizador **SGD por Adam**. ¿Qué configuración es la que mejores resultados proporciona sin mostrar signos de overfitting? 

In [0]:
# Imports necesarios
import numpy as np
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.optimizers import Adam
import matplotlib.pyplot as plt

# Importamos el dataset MNIST y cargamos los datos
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Implementamos la red empleando Keras
model = Sequential()
model.add(Dense(200, input_shape=(784,), activation="relu"))
#model.add(Dropout(0.5))
model.add(Dense(100, activation="relu"))
#model.add(Dropout(0.5))
model.add(Dense(60, activation="relu"))
#model.add(Dropout(0.5))
model.add(Dense(30, activation="relu"))
model.add(Dense(10, activation="softmax"))

# Comprobemos la estructura de la red
model.summary()

# Compilamos y entrenamos el modelo SGD. Sacamos el accuracy.
print("[INFO]: Entrenando red neuronal...")
model.compile(loss="categorical_crossentropy", optimizer=SGD(0.005), metrics=["accuracy"])
H = model.fit(mnist.train.images, mnist.train.labels, validation_data=(mnist.validation.images, mnist.validation.labels), epochs=50, batch_size=128)

# Evaluando el modelo de predicción con las imágenes de test
print("[INFO]: Evaluando red neuronal...")
predictions = model.predict(mnist.test.images, batch_size=128)
print(classification_report(mnist.test.labels.argmax(axis=1), predictions.argmax(axis=1)))

# Muestro gráfica de accuracy y losses
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 50), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 50), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 50), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 50), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()



---



**EJERCICIO 2.** A continuación vamos a trabajar con un dataset un poco más complejo, **[CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html)**. Dicho conjunto de datos se compone de **60000 imágenes RGB** de dimensiones **32x32** pertenecientes a **10 clases distintas** (6000 imágenes por clase). CIFAR10 se separa en dos subconjuntos de datos: **50000** imágenes para **entrenamiento** y las **10000** restantes se emplean como set de **test**. 


In [0]:
# Importando el set de datos CIFAR10
from keras.datasets import cifar10
from sklearn.preprocessing import LabelBinarizer
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
# Transformamos los valores de intensidad al rango 0-1
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0
# Etiquetas del dataset
labelNames = ["Avión", "Automóvil", "Pájaro", "Gato", "Ciervo", "Perro", "Rana", "Caballo", "Barco", "Camión"]
# Por si es necesario convertir a one-hot encoding
#lb = LabelBinarizer()
#trainY = lb.fit_transform(trainY)
#testY = lb.transform(testY)

Ahora que tenemos en memoria el set de datos CIFAR10, lo primero que debemos hacer es **mostrar unas cuantas imágenes** para visualizar la **variabilidad** existente:

In [0]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(14,10))
for n in range(1, 29):
    fig.add_subplot(4, 7, n)
    img = trainX[n]
    plt.imshow(img)
    plt.title(labelNames[trainY[n][0]])
    plt.axis('off')

1. Entrena una red perceptron multicapa con dos capas ocultas (la primera de ellas de **1024** neuronas y la segunda de **512**). Emplea como función de activación **ReLU** y **SGD** como optimizador con una tasa de aprendizaje **```lr = 0.01```**. Como función de pérdidas utilizaremos ***categorical crossentropy***. Entrenad la red con valores de **```epochs = 50```** y **```batch_size = 32```**.

  **Nota.** Prestad atención a como se cargan los datos, en caso de que se carguen  etiquetas en decimal se deberá emplear el método **```sparse_categorical_crossentropy```** o en su caso utilizar el objeto **LabelBinarizer** de la librería **ScikitLearn** para convertir las etiquetas a one hot encoding (i.e. etiquetas binarias) y poder emplear como función objetivo **```categorical_crossentropy```**. Busca en la documentación las diferencias entre ambos métodos y explicalas. Además el tipo de datos de nuestras etiquetas también hay que tenerlo en cuenta cuando hagamos uso del método **```classification_report```** para obtener las métricas de evaluación de nuestro modelo.

2. En caso de que la red no alcance una buena precisión prueba a dotarla de más profundidad. Concretamente incluye cinco capas ocultas con **2048, 1024, 512, 128 y 32** neuronas, respectivamente. Comenta los resultados comparando ambas arquitecturas de red. En caso de que se evidencien signos de overfitting realiza una **segunda ejecución de esta arquitectura** incluyendo una capa de **dropout** tras cada *hidden layer*. ¿Que sucede?



In [0]:
# Imports necesarios
import numpy as np
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
import matplotlib.pyplot as plt

# Pasamos los datos a vector con la función reshape
# ???
# ???

# Arquitectura de red
# Definimos el modo API Sequential
# ???
# Primera capa oculta
# ???
# Segunda capa oculta
# ???
# Tercera capa oculta
# ???
# Cuarta capa oculta
# ???
# Quinta capa oculta
# ???
# Capa de salida
# ???

# Compilamos el modelo sacando el accuracy y entrenamos
print("[INFO]: Entrenando red neuronal...")
# Compilamos el modelo
# ???

# Entrenamos el perceptrón multicapa
# ???

# Evaluamos con las muestras de test
print("[INFO]: Evaluando modelo...")
# Efectuamos predicciones
# ???
# Obtenemos el report (requiere etiquetas y predicciones categóricas)
# ???

# Mostramos gráfica de accuracy y losses
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 50), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 50), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 50), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 50), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()




---



Tal y como hemos visto en la sesión teórica, en la mayoría de problemas de clasificación de imagen, no es suficiente con crear un modelo de predicción basado en un perceptrón multicapa. Para problemas de cierta dificultad, este tipo de arquitectura no ofrece una solución precisa. Por este motivo se propusieron las **redes neuronales convolucionales**. Dichas arquitecturas de red **extraen la información relevante automáticamente** de la imagen por medio de la operación convolución de manera local (en la práctica dicha operación es la correlación cruzada). 

**EJERCICIO 3.** En el siguiente ejercicio vamos a desarrollar **nuestra primera red neuronal convolucional** y entrenarla sobre el **conjunto de datos CIFAR10**. Para ello, vamos a definir en el método ```shallow_CNN(width, heigh, depth, classes)```una arquitectura de red formada por **un único bloque convolucional** compuesto por una capa convolucional (**Conv2D** en Keras) de **32 filtros 3x3** empleando **```padding="same"```** y  la función de activación **ReLu**. Posteriormente, **estiraremos el volumen** resultante y lo llevaremos a una **capa de salida** compuesta por **10 neuronas**. Como se ha detallado en la sesión teórica, Keras tiene dos métodos distintos para implementar la arquitectura de red: el secuencial y el funcional. Implementa esta primera red convolucional utilizando el **método secuencial**. De nuevo emplea **SGD** como optimizador con una tasa de aprendizaje **```lr = 0.01```**. Como función de pérdidas utilizaremos ***categorical crossentropy*** (para etiquetas binarias o decimales según el caso). Entrenad la red con valores de **```epochs = 50```** y **```batch_size = 32```**. ¿Se produce una mejora sustancial con respecto a la arquitectura perceptrón multicapa? ¿Que crees que necesita nuestra CNN? 

**Nota.** Emplead el padding necesario para que las dimensiones del mapa de activación tras la capa convolucional se mantengan intactas.

In [0]:
# import the necessary packages
import numpy as np
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras import backend as K

def shallow_CNN(width, height, depth, classes):
  # Definimos el modo API Sequential y las dimensiones de la entrada (suponemos TF->"channels last")
  # ???
  # ???
  # Definir la arquitectura
  # Capa convolucional
  # ???
  # Clasificación
  # Estiramos el volumen de activación a un vector
  # ???
  # Añadimos capa de salida
  # ???
  # La función debe devolver el modelo como salida
  # ???

# Compilar el modelo
print("[INFO]: Compilando el modelo...")
# Instanciamos el modelo ajustado al dataset CIFAR10
# ???
# Compilamos el modelo sacando el accuracy
# ???

# Entrenamiento de la red
print("[INFO]: Entrenando la red...")
# ???

# Evaluación del modelo
print("[INFO]: Evaluando el modelo...")
# Efectuamos la predicción (empleamos el mismo valor de batch_size que en training)
# ???
# Sacamos el report para test
# ???

# Gráficas
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 50), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 50), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 50), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 50), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

De los resultados extraidos del apartado anterior se puede observar que para separar los datos de CIFAR10 en las distintas clases que lo componen requerimos de un **mayor número de bloques convolucionales**. Cuando esto sucede es necesario **introducir capas de pooling** entre bloques convolucionales sucesivos con el objetivo de reducir las dimensiones espaciales de un bloque al siguiente a la vez que aumenta el número de mapas de activación.

**EJERCICIO 4.** A continuación vamos a construir una **arquitectura** más avanzada **compuesta por dos bloques convolucionales (*base model*) y un bloque destinado a la clasificación (*top model*)** tal y como se muestra en la imagen siguiente:

![CNN_CIFAR10](https://drive.google.com/uc?id=1F6upnZDAp6su41bSegUJmRZgwXlsnqmY)


Codifica la arquitectura de la figura anterior en una función definida como:

   >>>```deep_CNN(width, height, depth, classes, batchNorm)```

El parámetro de entrada **```batchNorm```** debe ser una **bandera** a partir de la cual se aplique la técnica de **Batch Normalization** en los lugares indicados en la arquitectura en el caso que ```batchNorm=True```. En caso contrario no se aplicará dicha técnica. En este ejercicio **SE DEBE** emplear la **API funcional de Keras** para crear la arquitectura. Cabe destacar que en esta ocasión al tratarse de una red con mayor profundidad vamos a aplicar la técnica de **learning rate decay** y **nesterov acceleration** con valores **```decay=lr/epochs```** y **```momentum=0.9```**, respectivamente.  

- Ejecutad el entrenamiento con valores de **```epochs = 50```** y **```batch_size = 64```**. ¿Que se pueden decir ahora sobre los resultados de clasificación?¿Existe alguna diferencia entre la ejecución con ```batchNorm=True``` y ```batchNorm=False```?  

**Nota 1.** Emplear el padding necesario para que las dimensiones del mapa de activación tras cualquier capa convolucional se mantengan intactas.

**Nota 2.** La fase de **entrenamiento** en este ejercicio ya empieza a consumir un **tiempo considerable** de la sesión. Es por ello, que con tal de evitar que tengaís que volver a realizar el entrenamiento en caso de cierre insesperado del navegador o cualquier otro error **GUARDAD el modelo** una vez entrenado en un directorio de vuestro Google Drive (i.e. **/My Drive/Curso_CFP_DL/P4/Models**). Para montar Google Drive en vuestro código en Colab leed las trasparencias de la sesión teórica. Para almacenar el modelo entrenado emplead la fúncion de Keras **```mymodel.save(path)```** (no sin antes leer la documentación de la misma)


In [0]:
# import the necessary packages
import numpy as np
from keras import backend as K
from keras.layers.convolutional import Conv2D
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import MaxPooling2D
from keras.models import Sequential
from keras.optimizers import SGD
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from google.colab import drive

def deep_CNN(width, height, depth, classes, batchNorm):
  
  # Definimos entradas en modo "channels last"
  # ???
    
  # Definimos la arquitectura
  # Primer set de capas CONV => RELU => CONV => RELU => POOL
  # ???
  if batchNorm: 
    # ???
  # ???
  if batchNorm:
    # ???
  # ???
  # ???
  
  # Segundo set de capas CONV => RELU => CONV => RELU => POOL
  # ???
  if batchNorm:
    # ???
  # ???
  if batchNorm:
    # ???
  # ???
  # ???
  
  # Primer (y único) set de capas FC => RELU
  # ???
  # ???
  if batchNorm:
    # ???
  # ???
  # Clasificador softmax
  # ???
  
  # Unimos las entradas y el modelo mediante la función Model con parámetros inputs y ouputs (Consultar la documentación)
  # ???
  
  # La función debe devolver el modelo como salida           
  # ???

# Compilar el modelo
print("[INFO]: Compilando el modelo...")
# Instanciamos el modelo ajustado al dataset CIFAR10
# ???
# Compilamos el modelo sacando como mérica el accuracy
# ???

# Entrenamiento de la red
print("[INFO]: Entrenando la red...")
# ???

# Almaceno el modelo en Drive
# Montamos la unidad de Drive
# ???
# Almacenamos el modelo empleando la función mdoel.save de Keras
# ???

# Evaluación del modelo
print("[INFO]: Evaluando el modelo...")
# Efectuamos la predicción (empleamos el mismo valor de batch_size que en training)
# ???
# Sacamos el report para test
# ???

# Gráficas
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 50), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 50), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 50), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 50), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()

**EJERCICIO 5.** Por útlimo, vamos a **desarrollar un método** que a partir de un modelo de predicción ya entrenado y **una imagen de test**, muestre la misma por pantalla incluyendo **la clase a la que pertenece** y el **nivel de confianza** que ofrece el modelo en dicha predicción en el título de la figura. Para ello, comprobad si existe la variable que contenía el modelo anterior (```if 'model' not in locals():```) y en caso negativo cargad el modelo anteriormente almacenado en Drive empleando el comando de Keras **load_model(path)**. La cabecera de la función será la siguiente:

>>>>>> ```def predict_image(image, model, gt_str):```

Cabe destacar que el tercer parámetro de entrada **```gt_str```** es una **cadena de texto** que valdrá CIFAR10 en caso que estemos prediciendo imágenes de dicho set de datos (por defecto) y el String con el *ground-truth* en caso de que estemos prediciendo imágenes externas (siguiente ejercicio).
**Ejecutad** dicho método entre **10-15 veces** variando la imagen a testear de las del **conjunto de test de CIFAR10**. ¿Que puedes decir sobre el éxito en la predicción del modelo generado? 

In [0]:
import cv2
import matplotlib.pyplot as plt
import imutils
import numpy as np
from google.colab import drive
from keras.models import load_model

def predict_image(image, model, gt_str="CIFAR10"):
  # Creamos una copia en la variable output sobre la que mostraremos el resultado (comando image.copy())
  # ???
  # Expandimos las dimensiones de la variable image de (32, 32, 3) a (1, 32, 32, 3) con np.expand_dims
  # ???

  # Clasificación de la imagen empleando el modelo
  print("[INFO]: Clasificando imagen...")
  # Realizamos la predicción y la almacenamos en la variable proba
  # ???
  print(proba)
  # Nos quedamos con la clase que presente una probabilidad mayor y buscamos la etiqueta en el vector labelNames
  # ???
  # ???
  # ???
  # En caso que en la variable gt_str no me pasen el string "CIFAR10" es que me estan pasando el string con la etiqueta
  # Si ese es el caso almaceno el gt de ese String (esto nos valdrá para predecir imágenes que no sean del dataset CIFAR10)
  if gt_str != "CIFAR10":
    # ???

  # Mostrando imagen e información
  label = "Predicción: {} - Confianza: {:.2f}% - Ground Truth: {}".format(label, proba[0][idx] * 100, gt)
  plt.imshow(output)
  plt.title(label)
  plt.show()

# Escogemos una imagen de test al azar tal y como se hizo al principio de la práctica anterior
# para mostrar numeros de MNIST aleatoriamente (consultala si es necesario)
# ???
# ???
if 'model' not in locals():
  # Montamos la unidad de Drive
  # ???
  # Cargamos el modelo empleando la función load_model
  # ???
# Predecimos la imagen (llamando a predict_image) pasando como parámetros la imagen, el modelo y la cadena de texto correspondiente
# ???

A continuación, cread un directorio en vuestro Google Drive (i.e. **/My Drive/Curso_CFP_DL/P4/Imagenes**) y almacenad diferentes imágenes obtenidas bien de Internet o de vuestras colecciones de imágenes personales (unas 10 imágenes). Empleando OPENCV leed una imagen del directorio y efectuar la predicción empleando el método anterior. Ahora el ground truth no lo teneís directamente disponible por lo que deberéis modificar la función para pasarle un String que contenga la etiqueta solución.

Predecid las 10 imágenes almacenadas en vuestro Google Drive, ¿Que puedes decir sobre el **éxito en la predicción del modelo** generado sobre **imágenes NO pertenecientes** al dataset **CIFAR10**?¿A que cree que puede ser debido este fenómeno?¿Que soluciones deberiamos adoptar para mejorar la precisión en la clasificación?

In [0]:
from google.colab import drive

# Montamos la unidad de Drive
# ???

# Selecciono imagen y la leo con OPENCV
img_path = # ??? # Path de Drive donde tengo la imagen (incluido el nombre de la misma) #(X)
img_test = cv2.imread(img_path, cv2.IMREAD_COLOR) # Leo imagen con OPENCV
img_test = cv2.cvtColor(img_test,cv2.COLOR_BGR2RGB) # Por defecto la carga en BGR, la convierto a RGB

# Muestro información de la imagen y hago la predicción sacando resultados
print(img_test.shape)
plt.imshow(img_test)
plt.title('my picture')
plt.show()

# Pre-procesamos tal y como hicimos para la fase de entrenamiento con las muestras de CIFAR10 (normalizándola de 0.0 a 1.0)
# ???
# Re-escalamos la imagen al tamaño con el que fue entrenada la red (comando cv2.resize)
# ???
# Predecimos la imagen pasando como parámetros a la función predict_image: la imagen, el modelo y string con el GT
# ???